In [1]:
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

In [2]:
pok = pd.read_csv('pokemon_data.csv')

In [49]:
pok.columns = pok.columns.str.replace(' ', '')
pok

,Name,Type1,Type2,HP,Attack,Defense,Sp.Atk,Sp.Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...
795,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True
796,DiancieMega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True
797,HoopaHoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True
798,HoopaHoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True


# Which Pokémon has the highest HP?


In [269]:
pok.loc[pok['HP'].idxmax(), ['Name', 'HP']]

Name    Blissey
HP          255
Name: 261, dtype: object

# What is the average attack stat of Grass-type Pokémon?


In [54]:
pok.query("Type1 == 'Grass' or Type2 == 'Grass'")['Attack'].mean()

73.46315789473684

# Which Pokémon has the highest total base stats (sum of HP, Attack, Defense, Sp. Atk, Sp. Def, and Speed)?


In [270]:
pok['TotalBaseStats'] = pok.loc[:, 'HP':'Speed'].sum(axis=1)
pok.loc[pok['TotalBaseStats'].idxmax()]


Name              MewtwoMega Mewtwo X
Type1                         Psychic
Type2                        Fighting
HP                                106
Attack                            190
Defense                           100
Sp.Atk                            154
Sp.Def                            100
Speed                             130
Generation                          1
Legendary                        True
TotalBaseStats                    780
att_def_diff                       54
AtkDefRation                      1.9
Name: 163, dtype: object

# What is the correlation coefficient between the attack and defense stats?


In [95]:
pok['Attack'].corr(pok['Defense'])

0.4386870551184895

# Which generation has the most legendary Pokémon?


In [136]:
pd.DataFrame(pok[pok['Legendary'] == True].groupby('Generation')['TotalBaseStats'].count()).rename(columns={'TotalBaseStats': 'LegendCount'})

,LegendCount
Generation,
1,6
2,5
3,18
4,13
5,15
6,8


# What is the average HP of Water-type Pokémon with a speed stat greater than 80?


In [155]:
pok.query("(Type1 == 'Water' or Type2 == 'Water') and (Speed > 80)")['HP'].mean()

72.96875

# Among the Pokémon with a single type, which type combination has the highest average attack stat?


In [183]:
grouped = pok.query("@pd.isna(Type2)").groupby('Type1')['Attack'].describe()['mean']
print(grouped.idxmax(), '-',grouped.max())


Flying - 107.5


# What is the difference between the average speed of legendary Pokémon and non-legendary Pokémon?


In [191]:

lgnd = pok.query("Legendary == True")
nonlgnd = pok.query("Legendary == False")

lgnd['Speed'].mean() - nonlgnd['Speed'].mean()

34.72883307169022

# For each type combination (Type 1 + Type 2), what is the average defense stat?


In [204]:
pd.DataFrame(pok.groupby(by=['Type1', 'Type2'])['Defense'].mean())

Defense
Type1 Type2               
Bug   Electric   55.000000
      Fighting   95.000000
      Fire       60.000000
      Flying     61.571429
      Ghost      45.000000
...                    ...
Water Ice       113.333333
      Poison     58.333333
      Psychic   104.000000
      Rock      112.750000
      Steel      88.000000

[136 rows x 1 columns]

# Which Pokémon has the largest difference between its Special Attack and Special Defense stats?


In [219]:
pok['att_def_diff'] = pok['Sp.Atk'] - pok['Sp.Def']
idx = pok['att_def_diff'].idxmax()
pok.loc[idx]

Name              DeoxysAttack Forme
Type1                        Psychic
Type2                            NaN
HP                                50
Attack                           180
Defense                           20
Sp.Atk                           180
Sp.Def                            20
Speed                            150
Generation                         3
Legendary                       True
TotalBaseStats                   600
att_def_diff                     160
Name: 429, dtype: object

# Among the Pokémon with a base HP greater than 70, which one has the highest Attack stat?


In [222]:
pok.loc[pok.query("HP > 70")['Attack'].idxmax()]

Name              MewtwoMega Mewtwo X
Type1                         Psychic
Type2                        Fighting
HP                                106
Attack                            190
Defense                           100
Sp.Atk                            154
Sp.Def                            100
Speed                             130
Generation                          1
Legendary                        True
TotalBaseStats                    780
att_def_diff                       54
Name: 163, dtype: object

# How many Pokémon have a higher Special Attack stat than both their Attack and Defense stats combined?


In [233]:
pok[(pok['Sp.Atk'] > pok['Attack']) & (pok['Sp.Atk'] > pok['Defense'])].shape[0]

234

# What is the median speed of the Fire-type Pokémon in each generation?


In [252]:
pd.DataFrame(pok.query("Type1 == 'Fire' or Type2 == 'Fire'").groupby(by='Generation')['Speed'].median()).rename(columns={'Speed': 'MedianSpeed'})

,MedianSpeed
Generation,
1,91.5
2,83.0
3,45.0
4,82.0
5,64.5
6,78.5


# Among the Pokémon with a base HP between 50 and 70 (inclusive), which one has the highest sum of its Attack, Defense, and Speed stats?


In [254]:
grpd = pok.query("(HP > 50) and (HP < 70)")
grpd['SumStats'] = grpd['Attack'] + grpd['Defense'] + grpd['Speed']
idx = grpd['SumStats'].idxmax()
grpd.loc[idx]

/tmp/ipykernel_12616/855720277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grpd['SumStats'] = grpd['Attack'] + grpd['Defense'] + grpd['Speed']


Name              PinsirMega Pinsir
Type1                           Bug
Type2                        Flying
HP                               65
Attack                          155
Defense                         120
Sp.Atk                           65
Sp.Def                           90
Speed                           105
Generation                        1
Legendary                     False
TotalBaseStats                  600
att_def_diff                    -25
SumStats                        380
Name: 137, dtype: object

# Which Pokémon has the highest ratio of its Special Attack stat to its Special Defense stat?


In [259]:

pok['AtkDefRation'] = pok['Attack'] / pok['Defense']
idx = pok['AtkDefRation'].idxmax()
pok.loc[idx]

Name              DeoxysAttack Forme
Type1                        Psychic
Type2                            NaN
HP                                50
Attack                           180
Defense                           20
Sp.Atk                           180
Sp.Def                            20
Speed                            150
Generation                         3
Legendary                       True
TotalBaseStats                   600
att_def_diff                     160
AtkDefRation                     9.0
Name: 429, dtype: object

# How many Pokémon have a higher total base stat than the average total base stat of all Water-type Pokémon?


In [274]:

avg = pok.query("Type1 == 'Water' or Type2 == 'Water'")['TotalBaseStats'].mean()
pok.query("TotalBaseStats > @avg").shape[0]

428